In [1]:
import recommenders

from src.datasets import daocensus

%load_ext autoreload
%autoreload 2

In [2]:
SEED = 42
TOP_K = 5
EPOCHS = 50
BATCH_SIZE = 512

In [3]:
dfv, dfp = daocensus.get("./data/daos-census", 'Decentraland', 'snapshot', min_vpu=6)
print(dfv.info())
print(dfp.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107735 entries, 0 to 116559
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   platform       107735 non-null  object        
 1   name           107735 non-null  object        
 2   id             107735 non-null  object        
 3   proposal       107735 non-null  category      
 4   deployment     107735 non-null  object        
 5   platform_vote  107735 non-null  object        
 6   voter          107735 non-null  category      
 7   date           107735 non-null  datetime64[ns]
 8   choice         107735 non-null  object        
 9   weight         107735 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(1), object(6)
memory usage: 8.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1962 entries, 0 to 1961
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------    

In [4]:
def to_microsoft(dfv):
    df = dfv[['voter', 'proposal', 'date']].rename(columns={
        'voter': 'userID',
        'proposal': 'itemID',
        'date': 'timestamp',
    })
    df['itemID'] = df['itemID'].astype('str')
    df['rating'] = 1
    return df

df = to_microsoft(dfv)
df

,userID,itemID,timestamp,rating
0,0xe7af1c70f8f089c4c3bd71999692c6c5a15d9e2a,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 12:28:01,1
1,0xc54a6c3778016b06cbd126ccc3b5bc06c5f666fb,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 02:16:23,1
2,0xd82d005e8f8d5385db40ba23884a5c967bb1e8af,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 00:38:22,1
3,0xf4c64db66ffb301985f5ecd85c8f3f9c02f2659d,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:47:08,1
4,0xd5e9ef1cedad0d135d543d286a2c190b16cbb89e,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:32:15,1
...,...,...,...,...
116555,0x1156bf625b37623a86d004e66e0a01ec4b17e051,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:47:54,1
116556,0xbd03add5da0e173c67c9c1073ffba017147c42d4,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:44:04,1
116557,0x4da03f669dd9609dc49ca6f3451ba22d3b792395,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:21:14,1
116558,0xd6e62a97a55537cd04847bb73e22208bd20106aa,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:14:10,1


In [5]:
from recommenders.datasets.python_splitters import python_stratified_split

train, test = python_stratified_split(df, ratio=0.75)

In [6]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF

data = ImplicitCF(train=train, test=test, seed=SEED)

/home/davo/Documents/MUIA/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


In [7]:
data.n_items, data.n_users

(1941, 2900)

In [8]:
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

hparams = prepare_hparams(
    './config/lightgcn.yaml',
    n_layers=3,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    learning_rate=0.005,
    eval_epoch=5,
    top_k=TOP_K,
)

In [9]:
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN

model = LightGCN(hparams, data, seed=SEED)

2023-10-16 12:36:44.789111: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-16 12:36:44.790812: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 12:36:44.815049: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-16 12:36:44.815077: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-16 12:36:44.815108: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


2023-10-16 12:36:46.894514: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


In [10]:
model.fit()

Epoch 1 (train)2.6s: train loss = 0.37615 = (mf)0.37571 + (embed)0.00044
Epoch 2 (train)2.9s: train loss = 0.18560 = (mf)0.18452 + (embed)0.00108
Epoch 3 (train)2.9s: train loss = 0.14256 = (mf)0.14109 + (embed)0.00148
Epoch 4 (train)3.1s: train loss = 0.12065 = (mf)0.11888 + (embed)0.00177
Epoch 5 (train)2.6s + (eval)0.3s: train loss = 0.10370 = (mf)0.10167 + (embed)0.00203, recall = 0.16500, ndcg = 0.22976, precision = 0.19441, map = 0.11992
Epoch 6 (train)2.6s: train loss = 0.09391 = (mf)0.09163 + (embed)0.00227
Epoch 7 (train)2.6s: train loss = 0.08523 = (mf)0.08275 + (embed)0.00249
Epoch 8 (train)2.6s: train loss = 0.07906 = (mf)0.07637 + (embed)0.00269
Epoch 9 (train)2.6s: train loss = 0.07287 = (mf)0.06999 + (embed)0.00288
Epoch 10 (train)2.6s + (eval)0.3s: train loss = 0.06830 = (mf)0.06524 + (embed)0.00306, recall = 0.19378, ndcg = 0.26671, precision = 0.22552, map = 0.13851
Epoch 11 (train)2.6s: train loss = 0.06557 = (mf)0.06234 + (embed)0.00323
Epoch 12 (train)2.6s: train l

### Recommendation

We can call recommend_k_items to recommend k items for each user passed in this function. We set remove_seen=True to remove the items already seen by the user. The function returns a dataframe, containing each user and top k items recommended to them and the corresponding ranking scores.

In [11]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)
topk_scores.head()

,userID,itemID,prediction
0,0x06f3037a0f753c3ff2140f12a093e61245e5234f,5ed9c78f-0d56-5f11-b8aa-84b49c18e289,9.347280
1,0x06f3037a0f753c3ff2140f12a093e61245e5234f,ae73f53d-a5d1-522a-baf2-643b75ca481f,9.193292
2,0x06f3037a0f753c3ff2140f12a093e61245e5234f,bca2055a-d849-5308-b341-e8ce6b134ad8,9.131814
3,0x06f3037a0f753c3ff2140f12a093e61245e5234f,93337ffc-fb2e-536f-a194-0dca00990dbb,8.484897
4,0x06f3037a0f753c3ff2140f12a093e61245e5234f,877add37-cf0a-5420-84e6-ab010b814500,8.326041


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27028 entries, 101250 to 49375
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   userID     27028 non-null  category      
 1   itemID     27028 non-null  object        
 2   timestamp  27028 non-null  datetime64[ns]
 3   rating     27028 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1), object(1)
memory usage: 984.6+ KB


In [13]:
topk_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14500 entries, 0 to 14499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   userID      14500 non-null  category
 1   itemID      14500 non-null  object  
 2   prediction  14500 non-null  float32 
dtypes: category(1), float32(1), object(1)
memory usage: 285.6+ KB


### 2.5.2 Evaluation
With topk_scores predicted by the model, we can evaluate how LightGCN performs on this test set.

In [14]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      f"Precision@{TOP_K}:\t%f" % eval_precision,
      f"Recall@{TOP_K}:\t%f" % eval_recall, sep='\n')

MAP:	0.224801
NDCG:	0.388763
Precision@5:	0.317103
Recall@5:	0.294479


## 2.6 Infer embeddings

With infer_embedding method of LightGCN model, we can export the embeddings of users and items in the training set to CSV files for future use.

In [15]:
# model.infer_embedding(user_file, item_file)